# 강남 3구의 범죄현황 분석

- 강남3구의 주민들이 자신들이 거주하는 구의 체감 안전도를 높게 생각한다는 기사를 확인해 보도록 한다
- 기사 원문 http://news1.kr/articles/?1911504
- 작성 : PinkWink http://pinkwink.kr
  
### 
- Matplotlib의 heatmap 등을 그릴때 cmap의 디폴트 설정이 변경되어 heatmap 등에서 cmap을 적용할 때 옵션을 잡아주어야 교재와 동일한 효과가 나타난다. (소스코드에 모두 반영됨)
- Folium이 0.4.0으로 판올림 되면서 choropleth 명령에서 geo_str 옵션명이 geo_data 옵션명으로 변경됨. (소스코드에 모두 반영)
- Folium이 0.4.0으로 판올림 되면서 circle marker 적용할때, fill=True 옵션을 반듯이 사용해야 함. (소스코드에 모두 반영)

In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
b_path = 'C:\\Users\\user\\Documents\\Python\\data\\엘리스_데이터주무르기'

In [3]:
crime_path = os.path.join(b_path, '02. crime_in_Seoul.csv')
crime_anal_police = pd.read_csv(crime_path, thousands=',', 
                                encoding='euc-kr')
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


In [4]:
data_path = os.path.join(b_path, '39.4개시도_경찰관서현황.csv')
data = pd.read_csv(data_path)

In [5]:
data.head()

,name,psname,지구대파출소,X좌표,Y좌표,주소
0,경남청,진주경찰서,개양파출소,128.107802,35.156949,경상남도 진주시 가좌동 448-3
1,경남청,진주경찰서,금산파출소,128.148163,35.213726,경상남도 진주시 금산면 장사리 930-1
2,경남청,진주경찰서,남강지구대,128.072194,35.189701,경상남도 진주시 인사동 182-19
3,경남청,진주경찰서,대곡파출소,128.168509,35.262218,경상남도 진주시 대곡면 광석리 314-1
4,경남청,진주경찰서,문산파출소,128.164086,35.167648,경상남도 진주시 문산읍 삼곡리 1264-12


In [6]:
data = data[data['name'] == '서울청']
print(len(data))
data.head()

274


,name,psname,지구대파출소,X좌표,Y좌표,주소
53,서울청,서울강동경찰서,강일파출소,127.173245,37.564766,강동구 강일동 678-2
54,서울청,서울강동경찰서,광나루한강순찰대,127.118314,37.545966,서울시 강동구 천호동 488-1
55,서울청,서울강동경찰서,길동지구대,127.139201,37.529455,강동구 길동 228-6
56,서울청,서울강동경찰서,둔촌파출소,127.145744,37.525358,강동구 둔촌동 115
57,서울청,서울강동경찰서,명일파출소,127.156067,37.553640,강동구 명일동 47-20


In [7]:
data = data[data['지구대파출소'].str.contains('경찰서')]
print(len(data))
data.head()

31


,name,psname,지구대파출소,X좌표,Y좌표,주소
59,서울청,서울강동경찰서,서울강동경찰서,127.126971,37.528604,강동구 성내1동 540-1
64,서울청,서울강남경찰서,서울강남경찰서,127.067177,37.509036,강남구 대치동 998
73,서울청,서울송파경찰서,서울송파경찰서,127.126964,37.501853,송파구 가락본동 9
77,서울청,서울수서경찰서,서울수서경찰서,127.076976,37.493230,강남구 개포동 14
91,서울청,서울방배경찰서,서울방배경찰서,126.983018,37.481627,서초구 방배동 455-10


In [8]:
station_name = data['지구대파출소'].values.tolist()
station_name

['서울강동경찰서',
 '서울강남경찰서',
 '서울송파경찰서',
 '서울수서경찰서',
 '서울방배경찰서',
 '서울서초경찰서',
 '서울남대문경찰서',
 '서울서대문경찰서',
 '서울서부경찰서',
 '서울양천경찰서',
 '서울강서경찰서',
 '서울성북경찰서',
 '서울종암경찰서',
 '서울동작경찰서',
 '서울마포경찰서',
 '서울용산경찰서',
 '서울광진경찰서',
 '서울동대문경찰서',
 '서울성동경찰서',
 '서울중랑경찰서',
 '서울중부경찰서',
 '서울혜화경찰서',
 '서울종로경찰서',
 '서울관악경찰서',
 '서울금천경찰서',
 '서울구로경찰서',
 '서울영등포경찰서',
 '서울강북경찰서',
 '서울도봉경찰서',
 '서울은평경찰서',
 '서울노원경찰서']

In [9]:
station_addreess = data['주소'].values.tolist()
station_lat = data['Y좌표'].values.tolist()
station_lng = data['X좌표'].values.tolist()

In [10]:
gu_name = []

for name in station_addreess:
    tmp = name.split()
    
    tmp_gu = [gu for gu in tmp if gu[-1] == '구'][0]
    
    gu_name.append(tmp_gu)
    
crime_anal_police['구별'] = gu_name
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,강동구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,강남구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,송파구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,강남구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,서초구


In [11]:
crime_anal_police[crime_anal_police['관서명']=='금천서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
15,금천서,3,4,6,6,151,122,1567,888,2054,1776,용산구


In [12]:
crime_anal_police.loc[crime_anal_police['관서명']=='금천서', ['구별']] = '금천구'
crime_anal_police[crime_anal_police['관서명']=='금천서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
15,금천서,3,4,6,6,151,122,1567,888,2054,1776,금천구


In [13]:
crime_anal_path = os.path.join(b_path,'02. crime_in_Seoul_include_gu_name.csv')
crime_anal_police.to_csv(crime_anal_path,
                         sep=',', encoding='utf-8')
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,강동구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,강남구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,송파구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,강남구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,서초구


# pandas의 pivot_table 익히기

In [14]:
d_path = os.path.join(b_path, '02. sales-funnel.xlsx')
df = pd.read_excel(d_path)
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [15]:
pd.pivot_table(df,index=["Name"])

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.000000
"Fritsch, Russel and Anderson",737550,35000,1.000000
Herman LLC,141962,65000,2.000000
Jerde-Hilpert,412290,5000,2.000000
"Kassulke, Ondricka and Metz",307599,7000,3.000000
Keeling LLC,688981,100000,5.000000
Kiehn-Spinka,146832,65000,2.000000
Koepp Ltd,729833,35000,2.000000
Kulas Inc,218895,25000,1.500000


In [16]:
pd.pivot_table(df,index=["Name","Rep","Manager"])

,,,Account,Price,Quantity
Name,Rep,Manager,,,
Barton LLC,John Smith,Debra Henley,740150,35000,1.000000
"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,737550,35000,1.000000
Herman LLC,Cedric Moss,Fred Anderson,141962,65000,2.000000
Jerde-Hilpert,John Smith,Debra Henley,412290,5000,2.000000
"Kassulke, Ondricka and Metz",Wendy Yule,Fred Anderson,307599,7000,3.000000
Keeling LLC,Wendy Yule,Fred Anderson,688981,100000,5.000000
Kiehn-Spinka,Daniel Hilton,Debra Henley,146832,65000,2.000000
Koepp Ltd,Wendy Yule,Fred Anderson,729833,35000,2.000000
Kulas Inc,Daniel Hilton,Debra Henley,218895,25000,1.500000


In [17]:
pd.pivot_table(df,index=["Manager","Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

In [18]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

In [19]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [20]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

In [21]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

In [22]:
pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],
               columns=["Product"],aggfunc=[np.sum],fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [23]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],aggfunc=[np.sum],fill_value=0)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

In [24]:
pd.pivot_table(df,index=["Manager","Rep","Product"],
               values=["Price","Quantity"],
               aggfunc=[np.sum,np.mean],fill_value=0,margins=True)

sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

# 범죄 데이터 구별로 정리하기

In [25]:
crime_anal_raw_path = os.path.join(b_path,'02. crime_in_Seoul_include_gu_name.csv')
crime_anal_raw = pd.read_csv(crime_anal_raw_path, 
                             encoding='utf-8', index_col=0)
crime_anal_raw.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,강동구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,강남구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,송파구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,강남구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,서초구


In [26]:
crime_anal = pd.pivot_table(crime_anal_raw, index='구별', aggfunc=np.sum)
crime_anal.head()

,강간 검거,강간 발생,강도 검거,강도 발생,살인 검거,살인 발생,절도 검거,절도 발생,폭력 검거,폭력 발생
구별,,,,,,,,,,
강남구,222,269,9,11,5,5,1151,2882,2781,3334
강동구,65,105,2,3,2,2,477,1395,1170,1355
강북구,56,59,1,1,2,1,186,653,491,547
강서구,119,126,8,9,4,4,597,1607,1395,1612
관악구,193,334,5,8,4,7,905,1982,1607,1852


In [27]:
crime_anal['강간검거율'] = crime_anal['강간 검거']/crime_anal['강간 발생']*100
crime_anal['강도검거율'] = crime_anal['강도 검거']/crime_anal['강도 발생']*100
crime_anal['살인검거율'] = crime_anal['살인 검거']/crime_anal['살인 발생']*100
crime_anal['절도검거율'] = crime_anal['절도 검거']/crime_anal['절도 발생']*100
crime_anal['폭력검거율'] = crime_anal['폭력 검거']/crime_anal['폭력 발생']*100

del crime_anal['강간 검거']
del crime_anal['강도 검거']
del crime_anal['살인 검거']
del crime_anal['절도 검거']
del crime_anal['폭력 검거']

crime_anal.head()

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,269,11,5,2882,3334,82.527881,81.818182,100.000000,39.937543,83.413317
강동구,105,3,2,1395,1355,61.904762,66.666667,100.000000,34.193548,86.346863
강북구,59,1,1,653,547,94.915254,100.000000,200.000000,28.483920,89.762340
강서구,126,9,4,1607,1612,94.444444,88.888889,100.000000,37.149969,86.538462
관악구,334,8,7,1982,1852,57.784431,62.500000,57.142857,45.660949,86.771058


In [28]:
con_list = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']

for column in con_list:
    crime_anal.loc[crime_anal[column] > 100, column] = 100
    
crime_anal.head()

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,269,11,5,2882,3334,82.527881,81.818182,100.000000,39.937543,83.413317
강동구,105,3,2,1395,1355,61.904762,66.666667,100.000000,34.193548,86.346863
강북구,59,1,1,653,547,94.915254,100.000000,100.000000,28.483920,89.762340
강서구,126,9,4,1607,1612,94.444444,88.888889,100.000000,37.149969,86.538462
관악구,334,8,7,1982,1852,57.784431,62.500000,57.142857,45.660949,86.771058


In [32]:
crime_anal.rename(columns = {'강간 발생':'강간', 
                             '강도 발생':'강도', 
                             '살인 발생':'살인', 
                             '절도 발생':'절도', 
                             '폭력 발생':'폭력'}, inplace=True)
crime_anal.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,269,11,5,2882,3334,82.527881,81.818182,100.000000,39.937543,83.413317
강동구,105,3,2,1395,1355,61.904762,66.666667,100.000000,34.193548,86.346863
강북구,59,1,1,653,547,94.915254,100.000000,100.000000,28.483920,89.762340
강서구,126,9,4,1607,1612,94.444444,88.888889,100.000000,37.149969,86.538462
관악구,334,8,7,1982,1852,57.784431,62.500000,57.142857,45.660949,86.771058


In [34]:
from sklearn import preprocessing

col = ['강간', '강도', '살인', '절도', '폭력']

x = crime_anal[col].values
min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x.astype(float))
crime_anal_norm = pd.DataFrame(x_scaled, columns = col, index = crime_anal.index)

col2 = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']
crime_anal_norm[col2] = crime_anal[col2]
crime_anal_norm.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,0.450644,0.454545,0.307692,0.525956,0.693974,82.527881,81.818182,100.000000,39.937543,83.413317
강동구,0.098712,0.090909,0.076923,0.175083,0.201195,61.904762,66.666667,100.000000,34.193548,86.346863
강북구,0.000000,0.000000,0.000000,0.000000,0.000000,94.915254,100.000000,100.000000,28.483920,89.762340
강서구,0.143777,0.363636,0.230769,0.225106,0.265189,94.444444,88.888889,100.000000,37.149969,86.538462
관악구,0.590129,0.318182,0.461538,0.313591,0.324950,57.784431,62.500000,57.142857,45.660949,86.771058


In [36]:
x_scaled

array([[0.45064378, 0.45454545, 0.30769231, 0.52595564, 0.6939741 ],
       [0.09871245, 0.09090909, 0.07692308, 0.17508259, 0.20119522],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.14377682, 0.36363636, 0.23076923, 0.22510618, 0.26518924],
       [0.59012876, 0.31818182, 0.46153846, 0.31359132, 0.3249502 ],
       [0.27467811, 0.45454545, 0.92307692, 0.34969325, 0.57270916],
       [0.34549356, 0.54545455, 0.76923077, 0.61019349, 0.68426295],
       [0.45493562, 0.5       , 0.38461538, 0.66163285, 1.        ],
       [0.19313305, 0.22727273, 0.69230769, 0.18546484, 0.31673307],
       [0.07939914, 0.27272727, 0.        , 0.10429448, 0.22758964],
       [0.51716738, 0.63636364, 0.15384615, 0.41481831, 0.47758964],
       [0.02360515, 0.04545455, 0.07692308, 0.03916942, 0.16060757],
       [0.20171674, 0.59090909, 0.46153846, 0.18428504, 0.52340637],
       [0.24463519, 0.54545455, 0.30769231, 0.31335536, 0.49825697],
       [0.49570815, 0.81818182, 0.